<h1 align="center">Python Programming for Machine Learning</h1>
<h2 align="center">Case Studies</h2>

<center><img src='images/python-logo-only.svg' width=250> </center>

In [ ]:
import numpy as np
import torch
import matplotlib

print(f"NumPy: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")

Reminder:
- Lecturer: Have you started the recording?
- Audience: Have you received a notification that the recording has started?

This lecture:

- Walks through several "independent" concepts
- Some parts contain examples from real codebases

Optional background: [introductory OOP lecture from MIT](https://www.youtube.com/watch?v=-DP1i2ZU9gk)

> “Developers should always understand one layer of abstraction below their everyday work.”
>
> Glenn Vanderburg

# Iterables & Iterators

### 🟦 general example

Goal: implementing a dataset and dataloader "from scratch":
```python
dataset = Dataset([1, 2, 3, 4, 5, 6])
dataloader = Dataloader(dataset)

for epoch in range(2):
    for item in dataloader:
        print(item)
```

In [ ]:
# data container / iterable
class Dataset:
    def __init__(self, items):  # initialize instance: `dataset = Dataset(...)``
        self.items = items

    def __getitem__(self, idx):  # retrieve item: `dataset[1]``
        return self.items[idx]

    def __len__(self):  # get length: `len(dataset)`
        return len(self.items)

    # `__iter__` defines how to iterate
    # if undefined, Python falls back to simple sequential iteration
    # def __iter__(self):
    #     for cnt, item in enumerate(self.items):
    #         if cnt % 2:
    #             yield item


dataset = Dataset([1, 2, 3, 4, 5, 6])
dataset[1], len(dataset), "__iter__" in dir(dataset), [v for v in dataset]

Design choice: decouple dataset from dataloader (e.g., to use one dataloader for several datasets)

In [ ]:
# iterable
class Dataloader:
    def __init__(self, dataset):
        self.dataset = dataset

    def __iter__(self):
        for i in range(len(self.dataset)):
            yield self.dataset[i]  # `yield` creates iterator

        # More verbose alternative
        # return DataloaderIterator(self.dataset)


# class DataloaderIterator:
#     def __init__(self, dataset):
#         self.dataset = dataset
#         self.index = 0

#     def __iter__(self):
#         return self

#     # Defines "position" in dataset & gives next item
#     def __next__(self):
#         if self.index < len(self.dataset):
#             item = self.dataset[self.index]
#             self.index += 1
#             return item
#         raise StopIteration

In [ ]:
dataloader = Dataloader(dataset)

for epoch in range(2):
    print("new epoch")
    for item in dataloader:  # Each epoch gets fresh iterator
        print(item)

The above is syntactic sugar for the below:

In [ ]:
dataloader = Dataloader(dataset)

for epoch in range(2):

    # done by for loop behind the scenes
    # each call of `iter` returns own state (=iterator)
    iterator = iter(dataloader)

    while True:  # loop until the iterator is exhausted
        try:
            item = next(iterator)  # done by for loop behind the scenes
            print(item)
        except StopIteration:
            break

### 🟦 [openai/improved-diffusion](https://github.com/openai/improved-diffusion)

**Repo purpose:** diffusion models for images (see the diffusion model lecture)

**Application example**

Source: `improved_diffusion/image_dataset.py`
- Class `ImageDataset`

```python
from torch.utils.data import Dataset

class ImageDataset(Dataset)
    def __init__(self, resolution, image_paths, *args, classes=None, shard=0, num_shards=1, **kwargs):
        super().__init__(*args, **kwargs)
        self.resolution = resolution
        self.local_images = image_paths[shard:][::num_shards]
        self.local_classes = None if classes is None else classes[shard:][::num_shards]

    def __len__(self):
        return len(self.local_images)

    def __getitem__(self, idx):
        path = self.local_images[idx]
        with bf.BlobFile(path, "rb") as f:  # This is a context manager. More on this later.
            pil_image = Image.open(f)
            pil_image.load()

        # We may want preprocess data, e.g., standardize image aspect ratio
        [...]

        return np.transpose(arr, [2, 0, 1]), out_dict
```

### 🟦 [facebookresearch/chameleon](https://github.com/facebookresearch/chameleon)

**Repo purpose:** Mixed-modal models
- Similar to autoregressive language models, but support more modalities like text **and** images

**Application example:**

Source: `chameleon/inference/generation.py`

Class `ChameleonGenerator`: implements autoregressive (token by token) generation of the model via `__next__`

### Conclusions

- Iteration logic can be complex: not just dataloading: e.g. next-token prediction, streaming
- Several implementation tools: `__iter__`/`__next__`, generator functions, library helpers (e.g. dataloaders)
  - Generators: `__iter__` returns self, `__next__` runs to next `yield` (`return` → `StopIteration`)

> **💡 No single “best” tool: your job is to choose good abstractions for the context. This requires experience & expertise!**


# Overwriting / operator overloading & decorators/context managers

### 🟦 General example

**Overwriting:** replace (or extend via `super`) function via inheritance

In [ ]:
class Animal:
    def __init__(self, name: str):
        self.name = name
        print(f"Name: {name}")

    def move(self):
        print(f"{self.name} is moving")


class Bird(Animal):
    def __init__(self, wings: bool, *args, **kwargs):
        super().__init__(*args, **kwargs)  # super() points to parent class (→ we don't lose parent class logic)
        self.wings = wings

    def fly(self):
        print("Flying")


duck = Bird(wings=True, name="Daffy")
duck.fly()

### 🟦 general example

- Operator overloading (Python): same symbol (`+`) → type-dependent behavior
  - Implemented via dunder methods: `__add__`, `__radd__`, ...
- Function overloading (e.g., Java): same function name, many signatures: **not** relevant here

In [ ]:
class Dough:

    def __init__(self, weight=0.5, volume=0.5):
        self.weight = weight
        self.volume = volume

    def __repr__(self):
        return f"Dough(weight={self.weight}, volume={self.volume})"

    # operator overloading of `+`
    def __add__(self, other):  # instance_1 + instance_2
        # `type(self)` returns the class (`Dough`)
        return type(self)(self.weight + other.weight, self.volume + other.volume)
        # the below has identical behavior in *this* example, but may break when subclassing `Dough` without overwriting `__add__`
        # return Dough(self.weight + other.weight, self.volume + other.volume)

    # In-place addition (more memory efficient than `__add__`!)
    def __iadd__(self, other):  # instance_1 += instance_2
        self.weight += other.weight
        self.volume += other.volume
        return self

    def __mul__(self, other):  # instance_1 * instance_2
        return type(self)(self.weight * other.weight, self.volume * other.volume)

    def __rmul__(self, other):  # scalar * instance
        return type(self)(self.weight * other, self.volume * other)

In [ ]:
dough = Dough()

In [ ]:
dough  # calls __repr__

In [ ]:
dough += Dough(5, 5)  # calls __iadd__
dough

In [ ]:
3 * Dough(5, 5)  # calls __rmul__

In [ ]:
Dough(1) + Dough()  # calls __add__

In [ ]:
Dough() * Dough()  # calls __mul__

## Decorators & Context Managers

*Decorators were introduced in a previous lecture.*

### 🟦 General example

**Motivation**

- Build abstraction that:
  - performs computation before/after a function or code block
  - optionally also adjusts the function behavior/output
- Why abstraction (not: write this out "manually")?
  - Re-usable
  - Improved readability (factor out complexity)
  - Improved testability (test **only** this abstraction)

**Simple example**

`torch.no_grad` disables expensive gradient computation

Decorator:

In [ ]:

@torch.no_grad()  # decorator wraps function to disable gradient computation
def predict(model, data):  # noqa: F811
    # No gradient calculations happen anywhere in this function.
    return model(data)


Context manager:

In [ ]:

def predict(model, data):  # noqa: F811
    # Gradient computation is active
    with torch.no_grad():
        # Code in this block runs without gradient tracking.
        model(data)
    # Gradient computation is active

**How can we implement decorators & context managers ourself?**

Decorator function (most common, see a previous lecture):

In [ ]:
def function_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before function call")
        result = func(*args, **kwargs)
        print("After function call")
        return result
    return wrapper


@function_decorator
def my_function(x, y):
    print(f"Inside the function with arguments: x={x}, y={y}")
    return x / y

# Syntactic sugar for `my_function = function_decorator(my_function)`
my_function(5, 10)

Decorator class (useful for complex state):

In [ ]:
class ClassDecorator:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print("Before function call")
        result = self.func(*args, **kwargs)
        print("After function call")
        return result


@ClassDecorator
def my_function(x, y):
    print(f"Inside the function with arguments: x={x}, y={y}")
    return x / y


my_function(5, 10)  # This calls `MyDecorator.__call__`

`@contextmanager` implements a (limited) decorator **and** a context manager:
- can run code before/after a function & react to exceptions
- **cannot** change arguments, see/modify the return value, skip/delay the function

In [ ]:
from contextlib import contextmanager


@contextmanager
def contextlib_decorator():
    print("Before function call")
    try:
        # this is a generator function since it uses `yield`
        yield  # body or decorated functions runs here
    except Exception as e:
        print(f"Caught error: {e}")
        # optionally: raise
    finally:
        print("After function call")


Use as a decorator:

In [ ]:
@contextlib_decorator()  # `()` are necessary, even if the decorator has no arguments
def my_function(x, y):
    print(f"Inside the function with arguments: x={x}, y={y}")
    return x / y


my_function(5, 10)

Use as a context manager:

In [ ]:
def my_function(x, y):  # define without decorator
    print(f"Inside the function with arguments: x={x}, y={y}")
    return x + y


with contextlib_decorator():
    result = my_function(5, 10)

Not covered (in **this** lecture): decorators with function args (e.g., `@function_decorator(arg1, arg2)`) and context manager classes

### Conclusions

- Many tools for the job (e.g., not using a decorator/CM at all, different kinds of decorators/CMs)
- Similar to iterables/iterators:
  - which abstraction is suitable depends on the context
  - with experience/expertise, you will develop "taste" when to use which abstractions

## Combining operator overloading/overwriting & decorators

### 🟦 [google-deepmind/alphafold](https://github.com/google-deepmind/alphafold)
**Repo purpose:**
- Goal: amino acid sequences $\rightarrow$ 3D protein structures ([website](https://alphafold.ebi.ac.uk/))
- Hassabis, Jumper & Baker received the 2024 Chemistry Nobel for work that includes AlphaFold.

#### Application example: implementing geometries

- Domain-specific details (e.g., biology) & JAX details are **not** relevant for the exam (unless covered elsewhere).
- Python details are relevant.

**Decorator that creates a custom dataclass**

Source: `alphafold/model/geometry/struct_of_array.py`

Note: [dataclasses](https://www.youtube.com/watch?v=vBH6GRJ1REM) were introduced in a previous lecture

```python

class StructOfArray:

  def __init__(self, same_dtype=True):
    self.same_dtype = same_dtype

  # cls is the decorated class
  def __call__(self, cls):
    # [...]
    # monkey patching: changing object at runtime
    cls.__len__ = get_len
    cls.__getitem__ = get_item
    cls.__post_init__ = post_init

    # Classes decorated with `StructofArray` becomes a dataclass
    # `frozen=True`: dataclass is immutable → hashable → can be used as dict key
    new_cls = dataclasses.dataclass(cls, frozen=True, [...])

    # [...] JAX optimizations

    # decorator returns a new class that wraps the original class
    # this new class is initialized via `__init__`
    return new_cls
```

**Create optimized datastructure for 3D tensors & overload custom operators for specific geometries**

Source: `alphafold/model/geometry/vector.py`

```python
Float = float | jnp.ndarray  # type alias

@struct_of_array.StructOfArray(same_dtype=True)
class Vec3Array:
  """Vec3Array in 3 dimensional Space implemented as struct of arrays.

  This is done in order to improve performance and precision.
  [...]
  """

  # `jnp` is jax.numpy (not covered in this lecture)
  # "struct of arrays" –> separate variables for separate dimension
  # more efficient, b/c vector math is component-wise and components (e.g., x) are contiguous in mem (& JAX supports JIT-compilation to exploit this)
  x: jnp.ndarray = [...]
  y: jnp.ndarray
  z: jnp.ndarray

  [...]

  # Dunder methods below overload operators like `+` (= they define how these operators behave with `Vec3Array` types)

  def __add__(self, other: Vec3Array) -> Vec3Array:  # v1 + v2
    # `Vec3Array` is a PyTree (a JAX datastructure) with leaves (e.g., JAX arrays)
    # `tree_map` applies callable elementwise on leaves (x, y, z) of PyTrees `self` and `other`
    return jax.tree_map(
      lambda x, y: x + y,  # callable (function), `x + y` calls `__add__` of JAX arrays
      self,  # PyTree (this instance)
      other  # PyTree (`other`)
      )

  def __sub__(self, other: Vec3Array) -> Vec3Array:  # v1 - v2
    return jax.tree_map(lambda x, y: x - y, self, other)

  def __mul__(self, other: Float) -> Vec3Array:  # v1 * 3
    return jax.tree_map(lambda x: x * other, self)

  def __rmul__(self, other: Float) -> Vec3Array:  # 3 * v1
    return self * other

  def __truediv__(self, other: Float) -> Vec3Array:  # v1 / 2 (float division)
    return jax.tree_map(lambda x: x / other, self)

  def __neg__(self) -> Vec3Array:  # -v1
    return jax.tree_map(lambda x: -x, self)

  def __pos__(self) -> Vec3Array:  # +v1
    return jax.tree_map(lambda x: x, self)

  [...]

  # Similar to `torch.Tensor`, this class implements further non-dunder methods

  def dot(self, other: Vec3Array) -> Float:
    """Compute dot product between 'self' and 'other'."""
    return self.x * other.x + self.y * other.y + self.z * other.z

  def norm(self, epsilon: float = 1e-6) -> Float:
    [...]

  def norm2(self):
    [...]

  def normalized(self, epsilon: float = 1e-6) -> Vec3Array:
    [...]

  [...]
```

#### Application example: decorator
Source: `alphafold/data/tools/utils.py`

In [ ]:
import logging
import contextlib
import time

logging.basicConfig(level=logging.INFO, force=True)  # Ensure Jupyter outputs logs


@contextlib.contextmanager
def timer(msg: str):
  # More performant: This style defers string formatting until after the
  # log level is checked. An f-string is always evaluated, wasting
  # resources if the message is discarded.
  logging.info('Started %s', msg)

  tic = time.time()
  yield
  toc = time.time()
  logging.info('Finished %s in %.3f seconds', msg, toc - tic)

In [ ]:
@timer("Example Task")
def example_task():
    time.sleep(1.5)


example_task()

In [ ]:
def example_task():
    time.sleep(1.5)


with timer("Example Task"):
    example_task()

# Class vs. instance attributes/methods

**Motivation**

- Share one attribute (**one** memory address) with many instances
  - This lecture: use a static class attribute (less code, but more "global" & less flexible)
  - Alternative: pass the **same** object into each instance
- Build a "factory" (creates new instances)
  - This lecture: use `@classmethod` decorator (→ simpler API; easier subclassing)
  - Alternative: write a separate function

As above: which tool to use depends on the context!

### 🟦 General example

In [ ]:
def load_tokenizer(model_name: str):
    print(f"--- SLOW OPERATION: Loading '{model_name}' tokenizer from disk... ---")
    time.sleep(1)  # Simulate I/O delay
    return lambda text, max_len: f"Tokenized and truncated to {max_len}: '{text[:max_len]}...'"


class Processor:
    _tokenizer = None  # shared cache across multiple instances (*one* memory address used by *all* instances)

    @timer("init")
    def __init__(self, max_length: int):
        self.max_length = max_length
        self._ensure_tokenizer_loaded()
        # Replace the above line by the below line would reload the tokenizer once for every instance (*one* memory address for *each* instance)
        # self._tokenizer = load_tokenizer("4o-tokenizer")

    @classmethod
    # `cls` points to class (name by convention: the first arg points to class irrespective of how it's named)
    # type hint: Python resolves type later, `Processor` does not exist yet
    def _ensure_tokenizer_loaded(cls: "Processor"):
        if cls._tokenizer is None:
            # this happens (ideally) only once
            cls._tokenizer = load_tokenizer("4o-tokenizer")

    @timer("process")
    def process(self, text):
        # `type(self)` is `Processor` (flexible for subclassing).

        # `self._tokenizer` looks for instance attr (does not exist) & falls back to class attr
        # However: `_tokenizer`'s value is a function –> for `self.<function>`, Python passes `self` as first arg –> error
        # –> we use `<class>._tokenizer` instead

        return type(self)._tokenizer(text, self.max_length)

In [ ]:
processor1 = Processor(20)  # slow
processor1.process("Language modelling via autoregr. or diffusion models?")

In [ ]:
processor2 = Processor(25)  # fast
processor2.process("Student: \"Should I consider a PhD\"?")

We can swap out the class attr later:

In [ ]:
Processor._tokenizer = None

In [ ]:
processor2 = Processor(25)  # fast
processor2.process("Student: \"Should I consider a PhD\"?")

Relation to dataclasses:

In [ ]:
from dataclasses import dataclass


@dataclass
class Student:
    name: str
    university: str = "Berlin"  # class‐level default; treated as a field blueprint

# Decorator reads class vars (type + default) and implicitly generates:
#   def __init__(self, name, university="Berlin"):
#       self.name = name
#       self.university = university

In [ ]:
Student("Alice")

In [ ]:
Student("Bob", university="Munich")

### 🟦 [openai/improved-diffusion](https://github.com/openai/improved-diffusion): `improved_diffusion/logger.py`

All logger instances have global config:
```python
class Logger:
    DEFAULT = None  # allows swapping default logger globally
    CURRENT = None
```

### 🟦 [karpathy/nanochat](https://github.com/karpathy/nanochat): `scripts/tok_train.py` & `nanochat/tokenizer.py`


```python
class RustBPETokenizer:
    """Light wrapper around tiktoken (for efficient inference) but train with rustbpe"""

    @classmethod
    def train_from_iterator(cls, text_iterator, vocab_size):
        # 1) train using rustbpe
        tokenizer = rustbpe.Tokenizer()
        # the special tokens are inserted later in __init__, we don't train them here
        vocab_size_no_special = vocab_size - len(SPECIAL_TOKENS)
        assert vocab_size_no_special >= 256, f"vocab_size_no_special must be at least 256, got {vocab_size_no_special}"
        tokenizer.train_from_iterator(text_iterator, vocab_size_no_special, pattern=SPLIT_PATTERN)
        # 2) construct the associated tiktoken encoding for inference
        pattern = tokenizer.get_pattern()
        mergeable_ranks_list = tokenizer.get_mergeable_ranks()
        mergeable_ranks = {bytes(k): v for k, v in mergeable_ranks_list}
        tokens_offset = len(mergeable_ranks)
        special_tokens = {name: tokens_offset + i for i, name in enumerate(SPECIAL_TOKENS)}
        enc = tiktoken.Encoding(
            name="rustbpe",
            pat_str=pattern,
            mergeable_ranks=mergeable_ranks, # dict[bytes, int] (token bytes -> merge priority rank)
            special_tokens=special_tokens, # dict[str, int] (special token name -> token id)
        )
        return cls(enc, "<|bos|>")
```

# Inheritance

## Method Resolution Order
### 🟦 General example

How to deal with multiple parent classes:

In [ ]:
class A:
    def greet(self):
        return "Hello from A"


class B:
    def greet(self):
        return "Hello from B"


class C(A, B):
    pass


c = C()
print(c.greet())
print([cls.__name__ for cls in C.mro()])

> ⚠️ **Avoid deep inheritance trees**
> Code tightly coupled → difficult to debug

**Alternative: "has-a" instead of "is-a" relationships:**

"has-a" logic:

In [ ]:
class Engine:
    def start(self): ...


class Car:
    def __init__(self, engine: Engine):
        self.engine = engine   # has-a

    def drive(self):
        self.engine.start()

"is-a" logic:

In [ ]:
class Engine:
    def start(self): ...


class CarWithElectricEngine(Engine):
    def drive(self): ...

## Mixins
### 🟦 General example
Avoid complexity (overwriting) by using Mixins:

In [ ]:
class FlyMixin:
    def fly(self):
        print("Flying")


class SwimMixin:
    def swim(self):
        print("Swimming")


class Animal:
    def __init__(self, name: str):
        self.name = name


class Duck(Animal, FlyMixin, SwimMixin):
    pass


duck = Duck("Daffy")
duck.fly()
duck.swim()

### 🟦 [adapter-hub/adapters](https://github.com/adapter-hub/adapters)

**Repo purpose:** add-on library to HuggingFace's Transformers, e.g., for quantized training